In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
i = 0

In [4]:
matrisome_df = prep.load_matrisome_df(matrisome_list)

In [5]:
seed = 123
rand = np.random.RandomState()

# Load and filter survival data

In [6]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["figo_stage"]
cat_cols = ["race", "ethnicity"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[i]}/survival_data.tsv", event_code)

In [7]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="n")
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
        .pipe(prep.cols_to_front, ["sample_name", "figo_num"])
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')
print(filtered_survival_df.shape)
# filtered_survival_df.head()

(255, 12)


# Load normalized matrisome count data

In [8]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[i]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
# norm_filtered_matrisome_counts_t_df.head()

(255, 1009)


# Join survival and count data

In [9]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)
# joined_df.head()

(255, 1019)


# Optimize model

In [10]:
rand.seed(seed)
x_df, y_df = prep.shuffle_data(joined_df, rand)

In [11]:
def objective(h_params, X, y, r, verbose=True):
    if verbose:
        print(h_params)
    model = RandomForestClassifier(
        n_estimators=h_params[0],
        max_depth=h_params[1],
        max_features=h_params[2],
        min_samples_split=h_params[3],
        min_samples_leaf=h_params[4],
        bootstrap=h_params[5],
        n_jobs=-1,
        random_state=r
    )
    return -np.mean(cross_val_score(
        model,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring="f1_weighted"
    ))

In [12]:
space = [
    # We skip "ls" since we're optimizing for mean absolute error in the h_param opt.
    Integer(int(1e2), int(1e3), name="n_estimators"),
    Integer(int(10), int(100), name="max_depth"),
    Categorical(["auto", "sqrt", "log2"], name="max_features"),
    Integer(int(2), int(4), name="min_samples_split"),
    Integer(int(1), int(3), name="min_samples_leaf"),
    Categorical([True, False], name="bootstrap")
]
n_initial = 10 * len(space)
n_calls = 50 * len(space)

In [13]:
callback_file = f"{unified_dsets[i]}_opt_rfc_h_params.tsv"
try:
    os.remove(callback_file)
except OSError:
    pass


res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls = n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, callback_file, n = 5, sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
[171, 87, 'sqrt', 4, 1, False]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.9089
Function value obtained: -0.3913
Current minimum: -0.3913
Iteration No: 2 started. Evaluating function at random point.
[225, 46, 'log2', 3, 1, False]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.4787
Function value obtained: -0.3930
Current minimum: -0.3930
Iteration No: 3 started. Evaluating function at random point.
[830, 52, 'sqrt', 2, 2, True]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 4.4417
Function value obtained: -0.3699
Current minimum: -0.3930
Iteration No: 4 started. Evaluating function at random point.
[624, 29, 'log2', 3, 2, False]
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 2.6953
Function value obtained: -0.3704
Current minimum: -0.3930
Iteration No: 5 started. Evaluating function at random point.
[672, 13, 'log2', 3, 1, True]
Itera

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 75 ended. Search finished for the next optimal point.
Time taken: 2.8394
Function value obtained: -0.4060
Current minimum: -0.4113
Iteration No: 76 started. Searching for the next optimal point.
[100, 10, 'auto', 2, 1, False]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


*****redundant*****
Iteration No: 76 ended. Search finished for the next optimal point.
Time taken: 1.8579
Function value obtained: -0.3940
Current minimum: -0.4113
Iteration No: 77 started. Searching for the next optimal point.
[100, 10, 'auto', 4, 1, False]


...

*****redundant*****
Iteration No: 298 ended. Search finished for the next optimal point.
Time taken: 11.8899
Function value obtained: -0.4083
Current minimum: -0.4375
Iteration No: 299 started. Searching for the next optimal point.
[100, 10, 'sqrt', 2, 1, False]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


*****redundant*****
Iteration No: 299 ended. Search finished for the next optimal point.
Time taken: 11.0535
Function value obtained: -0.3812
Current minimum: -0.4375
Iteration No: 300 started. Searching for the next optimal point.
[100, 10, 'sqrt', 2, 1, False]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


*****redundant*****
Iteration No: 300 ended. Search finished for the next optimal point.
Time taken: 11.0020
Function value obtained: -0.3894
Current minimum: -0.4375
